<a href="https://colab.research.google.com/github/jhyoo78/jhyoo78/blob/main/Word_embedding_FastText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://wikidocs.net/22660 교재의 Word2vec 코드를 수정함

import numpy as np
import nltk
nltk.download('punkt')
import zipfile
import re
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import  stopwords                        # 향후 전처리로 to, for 등을 제거할 것

In [ ]:
# PC 에서 test1.txt(log file 이름)를 upload

from google.colab import files     # Colab에서 local PC내의 file 읽어 들이는 라이브러리
uploaded = files.upload()          # 파일을  선택

In [ ]:
# Google Drive를 이용할 경우에 실행
from google.colab import drive
drive.mount('./mount')

In [ ]:
f = open('test1.txt', 'r', encoding='UTF8')          # 1,100 줄의 log 파일
raw = f.read()

sent_text = sent_tokenize(raw)     #  log msg를 줄 단위로 sentence로 토큰화
print("sent_text==========", sent_text)

# 각 문장에 대해서 구두점, LF 등을 제거하고, 대문자를 소문자로 변환.
pre_processed_text = []
for string in sent_text:
    tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
    pre_processed_text.append(tokens)
print("pre_processed_text=", pre_processed_text)

# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
result = [word_tokenize(sentence) for sentence in pre_processed_text]

print('총 샘플의 개수 : {}'.format(len(result)))
print(result)


In [ ]:
# Word2Vec으로 sentence 학습시키기

from gensim.models import Word2Vec
model = Word2Vec(sentences = result, size=100, window=4, min_count=2, workers=4, sg=1)   # skip-gram 사용--> CBOW와 다른 결과 나옴


In [ ]:
model_result = model.wv.most_similar("system")
print(model_result)


In [ ]:
# Word2Vec 모델 저장하고 로드하기

from gensim.models import KeyedVectors
model.wv.save_word2vec_format('word2vec_log') # 모델 저장
loaded_model = KeyedVectors.load_word2vec_format("word2vec_log") # 모델 로드


In [ ]:
model_result1 = loaded_model.most_similar("failed")
print(model_result1)    #####  -----> 반의어의 유사도가 높게 나오는 문제 !!!!!!!!!!!!!!!! Word2Vec보다 FastText가 더 정확함.
model_result2 = loaded_model.most_similar("starting")
print(model_result2)

In [ ]:
# FastText로 훈련하기

from gensim.models import FastText
model = FastText(result, size=100, window=4, min_count=2, workers=4, sg=1)

model.save('fasttext_log')

In [ ]:
print(model['syste'])   # get the vector of the word 'syste'

In [ ]:
model.wv.most_similar('syste', topn=5)

In [ ]:
model.wv.most_similar(negative=['syste'], topn=5, )   # 반의어 찾기에 해당

In [ ]:
model.most_similar('fail', topn=5)

In [ ]:
model.most_similar('finished rotate log', topn=5)

In [ ]:
!apt-get install glove_python  # !pip도 build 오류 발생함, toy로 소개되어 있어서 ???

In [ ]:
# Glove로 훈련하기  --> import
from glove import Glove, Corpus
#from gensim.models import Glove, Corpus
corpus = Corpus()
corpus.fit(result, window=5)
# 훈련 데이터로부터 GloVe에서 사용할 동시 등장 행렬 생성

glove = Glove(no_components=100, learning_rate=0.05)
glove.fit(corpus.matrix, epochs=20, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)
# 학습에 이용할 쓰레드의 개수는 4로 설정, 에포크는 20.
